# Forsquare API data

## import libraries

In [1]:
import pandas as pd 
import requests
from pandas_profiling import ProfileReport

from geopy.geocoders import Nominatim 

# Maps visualization
import folium
import numpy as np
import plotly.express as px

import geopandas
import matplotlib.pyplot as plt

## Load data

In [2]:
df = pd.read_csv('Data/clean_london_data.csv')

In [3]:
df.head()

,Borough,Area (sq mi),Population_2019,Density,Median_Househols_Income,N_employees,latitude,longitude
0,Barnet,33.49,395896,"4,520","34,163","134,650",51.6252,0.1517
1,Bexley,23.38,248287,"4,082","29,192","78,930",51.4549,0.1505
2,Brent,16.70,329771,"7,652","28,847","123,260",51.5588,0.2817
3,Bromley,57.97,332336,"2,205","33,659","108,250",51.4039,0.0198
4,Camden,8.40,270029,"12,035","36,053","382,785",51.5290,0.1255


## Getting Venue Data with Forsquare API

In [4]:
CLIENT_ID = 'AIGPMW4KUAC351NMMXSGN4DPOTEXFTNBYSFV014YF3RE3FP5' # your Foursquare ID
CLIENT_SECRET = 'CY4SAG4M24EIB1NKVLUABAC2RUNNF53R0CBMKWTS3VUR5A3Q' # your Foursquare Secret
VERSION = '20220219'
LIMIT = 200

def getNearbyVenues(names, latitudes, longitudes, radius=2000,limit=200):

    URL= "https://api.foursquare.com/v3/places/nearby?ll={},{}&radius={}&limit=50&query=cafe"
    headers = {
        "Accept": "application/json",
        "Authorization": "fsq3Kdw1wp8+nAIcoOufls8EEFWG1ZTn43UcY0LQuY+zK30="
    }

    df_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = URL.format(lat, lng, radius, LIMIT)
        results = requests.request("GET", url, headers=headers).json()
        for each_result in results['results']: # filter the result based on JSON identification
            result={}
            result['Borough']=name
            result['Neighborhood Latitude']=lat
            result['Neighborhood Longitude']=lng
            result['Name']=each_result['name']
            result['Venue Latitude']=each_result['geocodes']['main']['latitude']
            result['Venue Longitude']=each_result['geocodes']['main']['longitude']


            #result['Locality']=each_result['location']['locality']
            result['Category_Names']=[each_name['name'] for each_name in each_result['categories']]


            df_list.append(result.copy())
    return pd.DataFrame(df_list) # return dataframe

In [5]:

# call the function
df_result=getNearbyVenues(df['Borough'],df['latitude'],df['longitude'])



> Here we want to check if we get venues from all the London boroughs.

In [6]:
df['Borough'].nunique()

29

In [7]:
df_result['Borough'].nunique()

29

## Merge the dataframes

In [8]:
# we will merge the dataframes relying on the Borough values
df = df.merge(df_result,on='Borough')


In [9]:
df.head()

,Borough,Area (sq mi),Population_2019,Density,Median_Househols_Income,N_employees,latitude,longitude,Neighborhood Latitude,Neighborhood Longitude,Name,Venue Latitude,Venue Longitude,Category_Names
0,Barnet,33.49,395896,"4,520","34,163","134,650",51.6252,0.1517,51.6252,0.1517,Café Kitsuné,51.499131,-0.157185,"[Coffee Shop, Restaurant]"
1,Barnet,33.49,395896,"4,520","34,163","134,650",51.6252,0.1517,51.6252,0.1517,Pavilion Cafe,51.533421,-0.042884,"[Café, Restaurant]"
2,Barnet,33.49,395896,"4,520","34,163","134,650",51.6252,0.1517,51.6252,0.1517,Cafe Oto,51.546786,-0.074763,"[Café, Coffee Shop, Japanese Restaurant]"
3,Barnet,33.49,395896,"4,520","34,163","134,650",51.6252,0.1517,51.6252,0.1517,French's Cafe,51.562863,0.220252,"[Café, Restaurant]"
4,Barnet,33.49,395896,"4,520","34,163","134,650",51.6252,0.1517,51.6252,0.1517,Dada Café,51.697793,0.110575,[Dining and Drinking]


In [13]:
df.to_csv("Data/final_data.csv", index = False)

## Visualising and Understanding the Data

In [14]:
from geopy.geocoders import Nominatim
nominatim_service = Nominatim(user_agent='sarah.a.alharbi@gmail.com')


# Read the GeoJSON file  
boroughs_json= r'london.json'

# Create a Map for London


london_coordinates = nominatim_service.geocode('London, UK')
london_map = folium.Map(location = [london_coordinates.latitude, london_coordinates.longitude], 
                        zoom_start=10)

# Create a Choropleth map for each Borough and respective population in 2010
london_map.choropleth(
    geo_data = boroughs_json,
    data =  df,
    columns = ['Borough','Population_2019'],
    key_on = 'feature.properties.name',
    fill_color="Blues",
    fill_opacity = 0.8,
    line_opacity = 0.8,
    legend_name = 'London Boroughs Population')

# In the below part, we create an overlay to add Popups to the map that displays 
# Borough name and Total Population

# Open Geo
# View the map
london_map



/usr/local/lib/python3.9/site-packages/folium/folium.py:409: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


FileNotFoundError: [Errno 2] No such file or directory: 'london.json'

In [15]:
 r'london.json'

'london.json'

In [16]:
nominatim_service = Nominatim(user_agent='sarah.a.alharbi@gmail.com')
# Create a Map for London


london_coordinates = nominatim_service.geocode('London, UK')
london_map = folium.Map(location = [london_coordinates.latitude, london_coordinates.longitude], 
                        zoom_start=10)

# Create a Choropleth map for each Borough and respective population in 2010
map1 = folium.Choropleth(
    geo_data = boroughs_json,
    data =  df,
    columns = ['Borough','Population_2019'],
    key_on = 'feature.properties.name',
    fill_color="YlGnBu",
    fill_opacity = 0.7,
    line_opacity = 0.8,
    reset=True,
    highlight=True,
    legend_name = 'London Boroughs Population').add_to(london_map)

# In the below part, we create an overlay to add Popups to the map that displays 
# Borough name and Total Population

# Open Geo
# View the map


map1.geojson.add_child(
    folium.features.GeoJsonTooltip(fields= ['name','pop_2001'],
                                   aliases=['Borough:', 'Population_2019:'],
                                   labels=True,
                                   localize=True,
                                   sticky=False,
                                   style="""
                                   background-color: #F0EFEF;
                                   border: 2px solid black;
                                   border-radius: 3px;
                                   box-shadow: 3px;
                                   """,)
)

london_map



FileNotFoundError: [Errno 2] No such file or directory: 'london.json'

### Issues:

- The population values in the geojson file is outdated since 2001, while in the scarpped data I've the popualtion values of 2019.

- The City of London is a city, ceremonial county and local government district that contains the historic centre and the primary central business district (CBD) of London.

In [ ]:
UK = json.load(open("london.json", "r"))


## Bar chart 

In [ ]:
# Create a Bar chart to plot each Borough's population

bar1 = london_boroughs.sort_values(by='Population_2019', ascending=False).plot(kind='bar', 
                                                         x='Borough', y='Population_2019', figsize=(15, 3));
# Title and axis labels
bar1.set_title("Borough Population in London (2019)");
bar1.set_xlabel("Borough");
bar1.set_ylabel("Total Population in 2019");



In [ ]:
# create bar chart of borough and number of cafes
bar2 = df.Borough.value_counts(ascending=False).plot(kind = 'bar',figsize=(15, 3), color='darkblue', rot= 25, linewidth = 4, edgecolor='white')                                   
# Title and axis labels
bar2.set_title("Borough Population in London (2019)");
bar2.set_xlabel("Borough" );
bar2.set_ylabel("Total Population in 2019");
#counts.sort_values(ascending=False).plot(kind='bar', color='darkblue')
#plt.xticks()

In [ ]:
df.Borough.value_counts()

In [ ]:
df.Category_Names.value_counts() 

# some of the cafe have different categories - encoding  

# Haven't Finished it yet




- Boroughs that we should consider, based on population count:

- Boroughs that we should consider, based on population density:


## Feature need to be added:
- Density population/Area
- create a list for Competitors  

## Want to achieve 

-  Create a Dataframe with top 5 common cafe types for each neighbourhood "tea room, with a restaurant, so on.. "
- Clustering Neighbourhoods

We will use KMeans algorithm in order to cluster similar London neighbourhoods. Then, explore how many competitors by category. 

Recommendation each boroughs with the category 



In [ ]:
df['Borough'] = df['Borough'].str.strip()


In [ ]:
df["id"] = df["Borough"].apply(lambda x: state_id_map[x])


In [ ]:
df.Borough = df.Borough.astype('string')


In [ ]:
state_id_map = {}
for feature in UK["name"]:
    feature["id"] = feature["properties"]["state_code"]
    state_id_map[feature["properties"]["st_nm"]] = feature["id"]




In [ ]:
fig = px.choropleth(
    df,
    locations="id",
    geojson= UK,
    color="Population_scale",
    hover_name="Borough",
    hover_data=["Population_scale"],
    title="London Population Density",
)
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

In [ ]:
df.dtypes